In [94]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [95]:
df = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
df.head()

In [96]:
df.shape

In [97]:
df.info()

In [98]:
vote_avg= df['vote_average'].mean()
#Truncate the upper %10 bound of top voted movies
vote_cnt= df['vote_count'].quantile(0.9) 
movies = df.copy().loc[df['vote_count'] >= vote_cnt]
movies.shape
print('The average vote rate is : ', vote_avg )
print('The total vote count is : ', vote_cnt )

In [99]:
def rating(x, vote_avg, vote_cnt):
    cnt = x['vote_count']
    avg = x['vote_average']
    return (cnt/(cnt+vote_cnt) * avg) + (vote_cnt/(vote_cnt+cnt) * vote_avg) 

In [100]:
movies['score'] = rating(movies,vote_avg,vote_cnt)

In [101]:
movies.head(5)

In [102]:
movies = movies.sort_values('score', ascending=False)
movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

In [103]:
movies['overview'].head(10)

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df = df.loc[0:10000,:]
#Filling nan values
df['overview'] = df['overview'].fillna('')

#Fit and transform
tfidf = TfidfVectorizer(stop_words='english').fit_transform(df['overview'])

In [105]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [114]:
def get_recommendations(title):
    
    idx = indices[title]
    cosine_sim = linear_kernel(tfidf, tfidf)
    sim_scores = list(enumerate(cosine_sim[idx]))
        
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].loc[movie_indices]

In [115]:
df['title'].head(5)

In [116]:
get_recommendations('Toy Story')